# Differential Privacy (DP) Private Aggregate Seeded Synthesizer

> Example based on [Synthetic Data Showcase - _pac-synth_](https://github.com/microsoft/synthetic-data-showcase/blob/main/packages/lib-pacsynth/samples/dp_aggregate_seeded_detailed_example.ipynb).

DP Private Aggregate Seeded synthesizer is a differentially private synthesizer that relies on DP Marginals to build synthetic data. It will compute DP Marginals (called aggregates) for your dataset using a specified `reporting length`, and synthesize data based on the computed aggregated counts.

> Aggregates will be computed for all lengths of attribute combination up to and including the `reporting length`.

## 1. Overview

### 1.1. Aggregate data generation with DP

Let's consider the following input as example:

| A   | B   | C   |
| --- | --- | --- |
| a1  | b1  | c1  |
| a1  | b2  | c1  |
| a2  |     | c2  |
| a2  | b2  | c1  |
| a1  | b2  |     |

The input data is assumed to be categorical and the domain will be inferred from the input dataset:

- `A` possible values are `a1,a2`
- `B` possible values are `b1,b2`
- `C` possible values are `c1,c2`

For a `reporting length=2`, the aggregates in the dataset above could be:

- 1-counts

  - `A:a1`: 3 + NOISE
  - `A:a2`: 2 + NOISE
  - `B:b1`: 1 + NOISE
  - `B:b2`: 3 + NOISE
  - `C:c1`: 3 + NOISE
  - `C:c2`: 1 + NOISE

- 2-counts:
  - `A:a1;C:c1`: 2 + NOISE
  - `A:a2;B:b2`: 1 + NOISE
  - `B:b1;C:c1`: 1 + NOISE
  - `A:a1;B:b1`: 1 + NOISE
  - `A:a1;B:b2`: 2 + NOISE
  - `B:b2;C:c1`: 2 + NOISE
  - `A:a2;C:c2`: 1 + NOISE
  - `B:b2;C:c2`: 0 + NOISE

Also, some spurious combinations might be created and reported to ensure differentially private guarantees - notice that `B:b2;C:c2` does not exist in the sensitive dataset, but it has been _fabricated_ and added to the output.

Similarly, some attribute combinations might be suppressed. For example, even though `A:a2;C:c1` exists in the sensitive dataset, it has not been reported as an aggregate.

### 1.2. Synthesis

Data will be then synthesized directly from the aggregates computed with differential privacy to produce synthetic data. Which will ensure the same DP guarantees to the synthetic data.


## 2. Imports and global config


In [1]:
import pandas as pd
import math

from snsynth.aggregate_seeded import (
    AggregateSeededSynthesizer,
    AccuracyMode,
    FabricationMode,
    AggregateSeededDataset,
)
from snsynth.transform.table import NoTransformer

from utils import gen_data_frame, ErrorReport


## 3. Generating an example data frame with random data

> `gen_data_frame` is just an utility to generate some example data (the code for it is in [`utils.py`](./utils.py))

To illustrate the library, let's start by creating an example data frame:


In [2]:
number_of_records_to_generate = 6000

sensitive_df = gen_data_frame(number_of_records_to_generate)

sensitive_df


,H1,H2,H3,H4,H5,H6,H7,H8,H9,H10
0,1,1,,1,0,1,1,1,0,0
1,2,1,,0,0,0,0,0,0,0
2,2,2,3,0,1,0,0,0,0,1
3,1,1,,0,1,0,1,0,1,1
4,1,1,4,1,0,0,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...
5995,1,5,7,0,0,1,0,1,1,0
5996,2,5,7,1,1,1,1,0,1,0
5997,2,4,,0,0,1,1,0,0,1
5998,1,5,10,1,0,1,1,0,0,0


## 4. Creating the sensitive dataset

### 4.1. Creation from constructor

The library uses an internal representation of the data to optimize execution time complexity.

If the data is already in the required raw format, you can call the constructor directly:


In [3]:
sensitive_raw_data = [
    # headers
    ["A", "B", "C", "D"],
    # records
    ["a1", "b1", "c1", "0"],
    ["a1", "b1", "c2", "0"],
    ["a2", "", "c2", "1"],
]
sensitive_dataset = AggregateSeededDataset(sensitive_raw_data)
sensitive_dataset.to_data_frame()


,A,B,C,D
0,a1,b1,c1,
1,a1,b1,c2,
2,a2,,c2,1


### 4.2. Negative value interpretation

The library distinguishes 'positive' attribute values that indicate the presence of specific sensitive data from 'negative' attribute values that indicate the absence of such data. By default, the integer zero (`0`) and the empty string (`""`) are not taken into account when creating and counting attribute combinations. Any columns where zero values are of interest (and thus sensitive) should be listed as `sensitive_zeros`, so they will be treated the same way as positive values.

> For more parameters see the library documentation - `help('pacsynth.Dataset')`.


In [4]:
sensitive_raw_data = [
    # headers
    ["A", "B", "C", "D"],
    # records
    ["a1", "b1", "c1", "0"],
    ["a1", "b1", "c2", "0"],
    ["a2", "", "c2", "1"],
]
sensitive_dataset = AggregateSeededDataset(sensitive_raw_data, sensitive_zeros=["D"])
sensitive_dataset.to_data_frame()


,A,B,C,D
0,a1,b1,c1,0
1,a1,b1,c2,0
2,a2,,c2,1


### 4.3. Creating sensitive dataset from a pandas data frame

For convenience, a method is provided to build a dataset from a pandas data frame:


In [5]:
sensitive_dataset = AggregateSeededDataset.from_data_frame(sensitive_df)
sensitive_dataset.to_data_frame()


,H1,H2,H3,H4,H5,H6,H7,H8,H9,H10
0,1,1,,1,,1,1,1,,
1,2,1,,,,,,,,
2,2,2,3,,1,,,,,1
3,1,1,,,1,,1,,1,1
4,1,1,4,1,,,1,1,1,
...,...,...,...,...,...,...,...,...,...,...
5995,1,5,7,,,1,,1,1,
5996,2,5,7,1,1,1,1,,1,
5997,2,4,,,,1,1,,,1
5998,1,5,10,1,,1,1,,,


## 5. Generating the synthetic data

### 5.1. Defining synthesizer parameters

If you just want to create the synthesizer with default parameters:


In [6]:
synth = AggregateSeededSynthesizer()
print(synth.parameters)


{
  "reporting_length": 3,
  "epsilon": 4.0,
  "delta": null,
  "percentile_percentage": 99,
  "percentile_epsilon_proportion": 0.01,
  "sigma_proportions": [
    1.0,
    0.5,
    0.3333333333333333
  ],
  "number_of_records_epsilon_proportion": 0.005,
  "threshold": {
    "type": "Adaptive",
    "valuesByLen": {
      "3": 1.0,
      "2": 1.0
    }
  },
  "empty_value": "",
  "use_synthetic_counts": false,
  "weight_selection_percentile": 95,
  "aggregate_counts_scale_factor": null
}


However, this might not produce the optimal output for your dataset and downstream analysis tasks.

So you can tune configuration by changing synthesizer parameters accordingly:


In [7]:
synth = AggregateSeededSynthesizer(
    reporting_length=3,
    epsilon=4.0,
    delta=1.0 / (math.log(len(sensitive_df)) * len(sensitive_df)),
    percentile_percentage=99,
    percentile_epsilon_proportion=0.01,
    accuracy_mode=AccuracyMode.prioritize_long_combinations(),
    number_of_records_epsilon_proportion=0.005,
    fabrication_mode=FabricationMode.uncontrolled(),
    empty_value="",
    weight_selection_percentile=95,
    use_synthetic_counts=False,
    aggregate_counts_scale_factor=1.0,
)
print(synth.parameters)


{
  "reporting_length": 3,
  "epsilon": 4.0,
  "delta": 0.000019158156689251674,
  "percentile_percentage": 99,
  "percentile_epsilon_proportion": 0.01,
  "sigma_proportions": [
    1.0,
    0.5,
    0.3333333333333333
  ],
  "number_of_records_epsilon_proportion": 0.005,
  "threshold": {
    "type": "Adaptive",
    "valuesByLen": {
      "3": 1.0,
      "2": 1.0
    }
  },
  "empty_value": "",
  "use_synthetic_counts": false,
  "weight_selection_percentile": 95,
  "aggregate_counts_scale_factor": 1.0
}


To continue with this example, let's set the parameters we care about for now:


In [8]:
reporting_length = 4

synth = AggregateSeededSynthesizer(
    reporting_length=reporting_length,
    epsilon=4.0,
    accuracy_mode=AccuracyMode.prioritize_long_combinations(),
    fabrication_mode=FabricationMode.uncontrolled(),
    use_synthetic_counts=True,
)


### 5.2 Building the model and synthesizing data


In [9]:
synth.fit(sensitive_dataset, transformer=NoTransformer())

# we could decide to use this or not as the sample number
protected_number_of_records = synth.get_dp_number_of_records()

print("Number of records protected with DP:", synth.get_dp_number_of_records())

# here if we do not specify the desired number of samples, the synthesizer will
# use all the available attributes based on the 1-counts to synthesize records
synthetic_raw_data = synth.sample(protected_number_of_records)
synthetic_dataset = AggregateSeededDataset(synthetic_raw_data)

# as an example, let's create a pandas data frame from the raw synthetic data
synthetic_df = AggregateSeededDataset.raw_data_to_data_frame(synthetic_raw_data)
synthetic_df


Number of records protected with DP: 5930


,H1,H2,H3,H4,H5,H6,H7,H8,H9,H10
0,,2,3,1,1,1,1,1,1,1
1,,2,3,1,1,1,1,1,1,1
2,,4,8,1,1,1,1,1,1,1
3,,4,8,1,1,1,1,1,1,1
4,,4,8,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...
5925,1,2,6,,,,,,,
5926,1,4,3,,,,,,,
5927,1,4,3,,,,,,,
5928,2,3,6,,,,,,,


## 6. Generating/exporting aggregate data


In [10]:
# generate sensitive aggregates
sensitive_aggregates = synth.get_sensitive_aggregates(";")

# export the differentially private aggregates (internal to the synthesizer)
dp_aggregates = synth.get_dp_aggregates(";")

# generate aggregates from the synthetic data
synthetic_aggregates = synthetic_dataset.get_aggregates(reporting_length, ";")

# let's take a look at the DP aggregates
list(dp_aggregates.items())[:20]


[('H2:6;H3:10;H5:1;H7:1', 27),
 ('H1:1;H3:6;H4:1;H8:1', 35),
 ('H1:1;H3:3;H6:1;H8:1', 27),
 ('H2:5;H3:9;H6:1;H8:1', 15),
 ('H1:1;H3:7;H6:1', 48),
 ('H1:2;H2:5;H3:9;H9:1', 17),
 ('H2:6;H3:9;H8:1', 54),
 ('H1:1;H2:1;H3:1;H4:1', 22),
 ('H10:1;H3:6;H8:1', 129),
 ('H1:2;H2:5;H4:1', 142),
 ('H10:1;H2:1;H3:3;H4:1', 68),
 ('H10:1;H2:1;H3:8', 8),
 ('H2:4;H3:8;H5:1;H9:1', 14),
 ('H10:1;H3:5;H4:1;H6:1', 92),
 ('H10:1;H1:1;H3:7;H8:1', 33),
 ('H2:1;H3:2;H6:1;H8:1', 20),
 ('H1:2;H2:4;H3:9;H9:1', 16),
 ('H1:2;H2:6;H3:7;H4:1', 29),
 ('H10:1;H1:2;H5:1;H6:1', 258),
 ('H10:1;H3:7;H5:1', 95)]

## 7. Evaluating DP aggregates and DP synthetic data

This section is an example evaluation of both the DP aggregates and the DP synthetic data, as well as the influence of some synthesizer parameters in the DP aggregates and synthetic data.


### 7.1. Evaluating current results

> `ErrorReport` is just an example way to evaluate results (the code for it is [`utils.py`](./utils.py))

- **Count**: mean of the aggregate counts for the given length
- **Error**: mean of the `abs(sensitive_count - dp_aggregated_count)` or `abs(sensitive_count - synthetic_count)`
- **Suppressed %**: percentage of combinations present in the sensitive dataset, but not present in the aggregated/synthetic data
- **Fabricated %**: percentage of combinations that were reported in the aggregated/synthetic data, but do not exist in the sensitive dataset


In [11]:
sensitive_aggregates_parsed = {
    tuple(agg.split(";")): count for (agg, count) in sensitive_aggregates.items()
}
dp_aggregates_parsed = {
    tuple(agg.split(";")): count for (agg, count) in dp_aggregates.items()
}
synthetic_aggregates_parsed = {
    tuple(agg.split(";")): count for (agg, count) in synthetic_aggregates.items()
}


**Sensitive Data vs. DP Aggregates**


In [12]:
ErrorReport(sensitive_aggregates_parsed, dp_aggregates_parsed).gen()


,Length,Count +/- Error,Suppressed %,Fabricated %
0,1,1375.80 +/- 15.36,0.00 %,0.00 %
1,2,422.80 +/- 17.93,0.00 %,6.28 %
2,3,147.59 +/- 18.72,0.44 %,7.00 %
3,4,55.40 +/- 14.75,8.00 %,2.70 %
4,All,110.22 +/- 16.03,5.53 %,4.11 %


**Sensitive Data vs. DP Synthetic Data**


In [13]:
ErrorReport(sensitive_aggregates_parsed, synthetic_aggregates_parsed).gen()


,Length,Count +/- Error,Suppressed %,Fabricated %
0,1,1375.80 +/- 99.76,0.00 %,0.00 %
1,2,422.80 +/- 69.91,0.00 %,6.28 %
2,3,147.59 +/- 39.59,0.44 %,6.23 %
3,4,55.40 +/- 19.16,8.21 %,2.32 %
4,All,110.22 +/- 28.50,5.67 %,3.65 %


### 7.2. Targeting less fabrication

Let's update the current synthesizer parameters to `minimize` fabrication:


In [14]:
synth = AggregateSeededSynthesizer(
    reporting_length=reporting_length,
    epsilon=4.0,
    accuracy_mode=AccuracyMode.prioritize_long_combinations(),
    fabrication_mode=FabricationMode.minimize(),
    use_synthetic_counts=True,
)

synth.fit(sensitive_dataset, transformer=NoTransformer())

synthetic_raw_data = synth.sample(protected_number_of_records)
synthetic_dataset = AggregateSeededDataset(synthetic_raw_data)

synthetic_df = AggregateSeededDataset.raw_data_to_data_frame(synthetic_raw_data)
synthetic_df


,H1,H2,H3,H4,H5,H6,H7,H8,H9,H10
0,1,,,1,1,1,1,1,1,1
1,1,,,1,1,1,1,1,1,1
2,1,,,1,1,1,1,1,1,1
3,1,,,1,1,1,1,1,1,1
4,1,,,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...
5925,2,6,6,,,,,,,
5926,2,6,6,,,,,,,
5927,2,6,6,,,,,,,
5928,2,6,6,,,,,,,


Evaluating again:


In [15]:
sensitive_aggregates = sensitive_dataset.get_aggregates(reporting_length, ";")
dp_aggregates = synth.get_dp_aggregates(";")
synthetic_aggregates = synthetic_dataset.get_aggregates(reporting_length, ";")

sensitive_aggregates_parsed = {
    tuple(agg.split(";")): count for (agg, count) in sensitive_aggregates.items()
}
dp_aggregates_parsed = {
    tuple(agg.split(";")): count for (agg, count) in dp_aggregates.items()
}
synthetic_aggregates_parsed = {
    tuple(agg.split(";")): count for (agg, count) in synthetic_aggregates.items()
}


**Sensitive Data vs. DP Aggregates**


In [16]:
ErrorReport(sensitive_aggregates_parsed, dp_aggregates_parsed).gen()


,Length,Count +/- Error,Suppressed %,Fabricated %
0,1,1375.80 +/- 19.36,0.00 %,0.00 %
1,2,422.80 +/- 18.14,0.00 %,0.00 %
2,3,147.59 +/- 20.06,23.04 %,0.00 %
3,4,55.40 +/- 17.04,69.12 %,0.00 %
4,All,110.22 +/- 18.48,52.69 %,0.00 %


**Sensitive Data vs. DP Synthetic Data**


In [17]:
ErrorReport(sensitive_aggregates_parsed, synthetic_aggregates_parsed).gen()


,Length,Count +/- Error,Suppressed %,Fabricated %
0,1,1375.80 +/- 283.72,0.00 %,0.00 %
1,2,422.80 +/- 168.77,0.00 %,0.00 %
2,3,147.59 +/- 90.37,23.04 %,0.00 %
3,4,55.40 +/- 45.19,69.12 %,0.00 %
4,All,110.22 +/- 83.13,52.69 %,0.00 %


### 7.3. Prioritize short combinations

Let's update the current synthesizer parameters to `prioritize_short_combinations`:


In [18]:
synth = AggregateSeededSynthesizer(
    reporting_length=reporting_length,
    epsilon=4.0,
    accuracy_mode=AccuracyMode.prioritize_short_combinations(),
    fabrication_mode=FabricationMode.uncontrolled(),
    use_synthetic_counts=False,
)

synth.fit(sensitive_dataset, transformer=NoTransformer())

synthetic_raw_data = synth.sample(protected_number_of_records)
synthetic_dataset = AggregateSeededDataset(synthetic_raw_data)

synthetic_df = AggregateSeededDataset.raw_data_to_data_frame(synthetic_raw_data)
synthetic_df


,H1,H2,H3,H4,H5,H6,H7,H8,H9,H10
0,1,,,1,1,1,1,1,1,1
1,1,,,1,1,1,1,1,1,1
2,1,,,1,1,1,1,1,1,1
3,1,,,1,1,1,1,1,1,1
4,1,,,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...
5925,,6,4,,,,,,,
5926,,6,7,,,,,,,
5927,,6,8,,,,,,,
5928,,6,9,,,,,,,


Evaluating again:


In [19]:
sensitive_aggregates = sensitive_dataset.get_aggregates(reporting_length, ";")
dp_aggregates = synth.get_dp_aggregates(";")
synthetic_aggregates = synthetic_dataset.get_aggregates(reporting_length, ";")

sensitive_aggregates_parsed = {
    tuple(agg.split(";")): count for (agg, count) in sensitive_aggregates.items()
}
dp_aggregates_parsed = {
    tuple(agg.split(";")): count for (agg, count) in dp_aggregates.items()
}
synthetic_aggregates_parsed = {
    tuple(agg.split(";")): count for (agg, count) in synthetic_aggregates.items()
}


**Sensitive Data vs. DP Aggregates**


In [20]:
ErrorReport(sensitive_aggregates_parsed, dp_aggregates_parsed).gen()


,Length,Count +/- Error,Suppressed %,Fabricated %
0,1,1375.80 +/- 3.08,0.00 %,0.00 %
1,2,422.80 +/- 12.16,0.00 %,5.86 %
2,3,147.59 +/- 29.16,1.65 %,6.79 %
3,4,55.40 +/- 27.88,33.60 %,4.00 %
4,All,110.22 +/- 26.86,23.16 %,5.04 %


**Sensitive Data vs. DP Synthetic Data**


In [21]:
ErrorReport(sensitive_aggregates_parsed, synthetic_aggregates_parsed).gen()


,Length,Count +/- Error,Suppressed %,Fabricated %
0,1,1375.80 +/- 88.48,0.00 %,0.00 %
1,2,422.80 +/- 117.15,0.00 %,5.43 %
2,3,147.59 +/- 80.07,1.87 %,5.62 %
3,4,55.40 +/- 52.32,35.44 %,3.08 %
4,All,110.22 +/- 67.01,24.46 %,4.10 %
